In [1]:
import cudf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from pandas_profiling import ProfileReport
pd.options.display.max_columns = None

In [2]:
sys.path.append(os.path.abspath(".."))
from src.utils import plot_venn2

In [3]:
train = cudf.read_feather("../input/train.ftr")
train.head()

/home/karunru/.pyenv/versions/miniconda3-4.7.12/envs/rapids-0.16/lib/python3.7/site-packages/cudf/io/feather.py:16: UserWarning: Using CPU via PyArrow to read feather dataset, this may be GPU accelerated in the future
  "Using CPU via PyArrow to read feather dataset, this may "


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,LEGO Batman: The Videogame,Wii,<NA>,Action,Warner Bros. Interactive Entertainment,180,97,0,28,306,74.0,17.0,7.9,22.0,Traveller's Tales,E10+
1,LEGO Indiana Jones: The Original Adventures,Wii,<NA>,Action,LucasArts,151,61,0,21,234,78.0,22.0,6.6,28.0,Traveller's Tales,E10+
2,LEGO Batman: The Videogame,PSP,<NA>,Action,Warner Bros. Interactive Entertainment,56,44,0,27,128,73.0,5.0,7.4,10.0,Traveller's Tales,E10+
3,Combat,2600,<NA>,Action,Atari,117,7,0,1,125,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,LEGO Harry Potter: Years 5-7,Wii,<NA>,Action,Warner Bros. Interactive Entertainment,69,42,0,12,124,76.0,8.0,7.8,13.0,Traveller's Tales,E10+


In [4]:
test = cudf.read_feather("../input/test.ftr")
test.head()

/home/karunru/.pyenv/versions/miniconda3-4.7.12/envs/rapids-0.16/lib/python3.7/site-packages/cudf/io/feather.py:16: UserWarning: Using CPU via PyArrow to read feather dataset, this may be GPU accelerated in the future
  "Using CPU via PyArrow to read feather dataset, this may "


,Name,Platform,Year_of_Release,Genre,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Hitman 2: Silent Assassin,XB,<NA>,Action,Eidos Interactive,84.0,23.0,8,19.0,Io Interactive,M
1,Legacy of Kain: Soul Reaver,PS,<NA>,Action,Eidos Interactive,91.0,17.0,9,132.0,Crystal Dynamics,T
2,Metal Gear Solid 2: Substance,XB,<NA>,Action,Konami Digital Entertainment,87.0,28.0,8.5,39.0,KCEJ,M
3,Silent Hill: Homecoming,X360,<NA>,Action,Konami Digital Entertainment,70.0,54.0,6.9,180.0,Double Helix Games,M
4,Silent Hill: Homecoming,PS3,<NA>,Action,Konami Digital Entertainment,71.0,41.0,6.9,143.0,Double Helix Games,M


In [6]:
total = cudf.concat([train, test], axis="index")
total.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,LEGO Batman: The Videogame,Wii,<NA>,Action,Warner Bros. Interactive Entertainment,180,97,0,28,306,74.0,17.0,7.9,22.0,Traveller's Tales,E10+
1,LEGO Indiana Jones: The Original Adventures,Wii,<NA>,Action,LucasArts,151,61,0,21,234,78.0,22.0,6.6,28.0,Traveller's Tales,E10+
2,LEGO Batman: The Videogame,PSP,<NA>,Action,Warner Bros. Interactive Entertainment,56,44,0,27,128,73.0,5.0,7.4,10.0,Traveller's Tales,E10+
3,Combat,2600,<NA>,Action,Atari,117,7,0,1,125,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,LEGO Harry Potter: Years 5-7,Wii,<NA>,Action,Warner Bros. Interactive Entertainment,69,42,0,12,124,76.0,8.0,7.8,13.0,Traveller's Tales,E10+


In [37]:
count_platforms_groupby_publisher = total.to_pandas().pivot_table(index='Publisher', columns='Platform',values='Name', aggfunc='count').reset_index()

In [38]:
count_platforms_groupby_publisher.columns = ["Publisher"] + ["count_platform_" + col  + "_groupby_publisher" for col in count_platforms_groupby_publisher.columns if col != "Publisher"]

In [43]:
count_platforms_groupby_publisher

,Publisher,count_platform_2600_groupby_publisher,count_platform_3DO_groupby_publisher,count_platform_3DS_groupby_publisher,count_platform_DC_groupby_publisher,count_platform_DS_groupby_publisher,count_platform_GB_groupby_publisher,count_platform_GBA_groupby_publisher,count_platform_GC_groupby_publisher,count_platform_GEN_groupby_publisher,count_platform_GG_groupby_publisher,count_platform_N64_groupby_publisher,count_platform_NES_groupby_publisher,count_platform_NG_groupby_publisher,count_platform_PC_groupby_publisher,count_platform_PCFX_groupby_publisher,count_platform_PS_groupby_publisher,count_platform_PS2_groupby_publisher,count_platform_PS3_groupby_publisher,count_platform_PS4_groupby_publisher,count_platform_PSP_groupby_publisher,count_platform_PSV_groupby_publisher,count_platform_SAT_groupby_publisher,count_platform_SCD_groupby_publisher,count_platform_SNES_groupby_publisher,count_platform_TG16_groupby_publisher,count_platform_WS_groupby_publisher,count_platform_Wii_groupby_publisher,count_platform_WiiU_groupby_publisher,count_platform_X360_groupby_publisher,count_platform_XB_groupby_publisher,count_platform_XOne_groupby_publisher
0,10TACLE Studios,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1C Company,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20th Century Fox Video Games,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2D Boy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3DO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,16.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,id Software,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
577,imageepoch Inc.,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
578,inXile Entertainment,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
579,"mixi, Inc",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
total.to_pandas().pivot_table(index='Publisher', columns='Genre',values='Name', aggfunc='count')

Genre,Action,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
Publisher,,,,,,,,,,,,
10TACLE Studios,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0
1C Company,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0
20th Century Fox Video Games,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
2D Boy,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3DO,17.0,3.0,1.0,NaN,1.0,1.0,NaN,1.0,5.0,NaN,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
id Software,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
imageepoch Inc.,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
inXile Entertainment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [11]:
total.to_pandas().pivot_table(index='Publisher', columns='Year_of_Release',values='Name', aggfunc='count')

Year_of_Release,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2020.0
Publisher,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10TACLE Studios,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1C Company,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20th Century Fox Video Games,NaN,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2D Boy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3DO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,6.0,10.0,11.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id Software,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
imageepoch Inc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
inXile Entertainment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [22]:
total.to_pandas().pivot_table(index='Publisher', columns='Rating',values='Name', aggfunc='count')

Rating,AO,E,E10+,EC,K-A,M,RP,T
Publisher,,,,,,,,
10TACLE Studios,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1C Company,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2D Boy,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3DO,NaN,5.0,NaN,NaN,NaN,NaN,NaN,13.0
505 Games,NaN,76.0,21.0,NaN,NaN,17.0,NaN,44.0
...,...,...,...,...,...,...,...,...
Zushi Games,NaN,8.0,4.0,NaN,NaN,NaN,NaN,3.0
bitComposer Games,NaN,1.0,NaN,NaN,NaN,NaN,NaN,3.0
iWin,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
total.groupby("Publisher")["Name"].count().reset_index()

,Publisher,Name
0,10TACLE Studios,3
1,1C Company,3
2,20th Century Fox Video Games,5
3,2D Boy,1
4,3DO,36
...,...,...
576,id Software,1
577,imageepoch Inc.,2
578,inXile Entertainment,1
579,"mixi, Inc",1


In [19]:
total.groupby("Publisher")["Name"].nunique().reset_index()

,Publisher,Name
0,10TACLE Studios,3
1,1C Company,3
2,20th Century Fox Video Games,5
3,2D Boy,1
4,3DO,30
...,...,...
576,id Software,1
577,imageepoch Inc.,2
578,inXile Entertainment,1
579,"mixi, Inc",1


In [14]:
total.groupby("Publisher")["Platform"].nunique().reset_index()

,Publisher,Platform
0,10TACLE Studios,2
1,1C Company,1
2,20th Century Fox Video Games,1
3,2D Boy,1
4,3DO,5
...,...,...
576,id Software,1
577,imageepoch Inc.,1
578,inXile Entertainment,1
579,"mixi, Inc",1


In [18]:
total.groupby("Publisher")["Genre"].nunique().reset_index()

,Publisher,Genre
0,10TACLE Studios,3
1,1C Company,3
2,20th Century Fox Video Games,2
3,2D Boy,1
4,3DO,9
...,...,...
576,id Software,1
577,imageepoch Inc.,2
578,inXile Entertainment,1
579,"mixi, Inc",1


In [15]:
total.groupby("Publisher")["Year_of_Release"].min().reset_index()

,Publisher,Year_of_Release
0,10TACLE Studios,2006.0
1,1C Company,2009.0
2,20th Century Fox Video Games,1981.0
3,2D Boy,2008.0
4,3DO,1998.0
...,...,...
576,id Software,1992.0
577,imageepoch Inc.,2014.0
578,inXile Entertainment,2015.0
579,"mixi, Inc",2015.0


In [16]:
total.groupby("Publisher")["Year_of_Release"].max().reset_index()

,Publisher,Year_of_Release
0,10TACLE Studios,2007.0
1,1C Company,2011.0
2,20th Century Fox Video Games,1982.0
3,2D Boy,2008.0
4,3DO,2003.0
...,...,...
576,id Software,1992.0
577,imageepoch Inc.,2014.0
578,inXile Entertainment,2015.0
579,"mixi, Inc",2015.0


In [23]:
total.groupby("Publisher")["Rating"].nunique().reset_index()

,Publisher,Rating
0,10TACLE Studios,1
1,1C Company,1
2,20th Century Fox Video Games,0
3,2D Boy,1
4,3DO,2
...,...,...
576,id Software,1
577,imageepoch Inc.,0
578,inXile Entertainment,1
579,"mixi, Inc",0


In [24]:
total.groupby("Publisher")["Developer"].nunique().reset_index()

,Publisher,Developer
0,10TACLE Studios,1
1,1C Company,3
2,20th Century Fox Video Games,0
3,2D Boy,1
4,3DO,4
...,...,...
576,id Software,1
577,imageepoch Inc.,0
578,inXile Entertainment,1
579,"mixi, Inc",0
